In [17]:
from mods import readdata as rd
from mods import savedata as sd
from mods import extractdata as ed
from mods import gmap as gm
from dotenv import load_dotenv
import os
import numpy as np

In [10]:
# 讀取raw data
path = "../../../data/raw/pet_establishment.csv"
df = rd.get_csv_data(path)
df

[✓] CSV檔案已取回



,ID,legaltype,legalname,legaladdress,busitem,animaltype,validnum,validdate,own_name,bos_name,rank_year,rank_code,rank_flag_1,rank_flag_2,rank_text,state_flag
0,15913,A010,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,特寵業字第V1120348號,2027/11/17 上午 12:00:00,宣家瑩,許玲玲,2024.0,B,Y,N,NaN,N
1,19029,A090,啟豐商行,桃園市桃園區文中一路25巷39號,AB,貓,特寵業繁字第C1140733號,2028/10/2 上午 12:00:00,黃韻蓁,袁子榕,NaN,NaN,NaN,NaN,NaN,N
2,19023,A210,偶然貓舍,臺南市永康區二王里41鄰南工街212巷3弄13號,ABC,貓,特寵業繁字第U1140846號,2028/10/7 上午 12:00:00,施寶美,劉品艾,NaN,NaN,NaN,NaN,NaN,N
3,17813,A060,沐光貓舍,臺中市南區工學北路357號十樓之3,ABC,貓,特寵業繁字第S1130016號,2027/3/14 上午 12:00:00,許小芸,許芸鳳、黃信捷,NaN,NaN,NaN,NaN,NaN,N
4,19041,A020,森光貓境貓舍,新北市板橋區雙十路2段46巷22弄5號1樓,ABC,貓,特寵業繁字第A1140069號,2028/10/19 上午 12:00:00,洪蔡佳芬,周宜潔,NaN,NaN,NaN,NaN,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5499,17,A040,澎澎犬舍,高雄市旗山區大福巷5號,AB,狗,特寵業繁字第W1080095號,2022/1/10 上午 12:00:00,張志雄,彭秀惠,2019.0,B,Y,N,NaN,B
5500,6,A040,東威犬舍,高雄市楠梓區藍昌路51號,ABC,狗,特寵業繁字第W1080128號,2022/3/14 上午 12:00:00,詹東旺,曾健倫,2019.0,B,Y,N,NaN,P
5501,4,A040,台灣犬研育場,高雄市鳥松區神農路335-1號,ABC,狗,特寵業繁字第W1070135號,2021/11/16 上午 12:00:00,周張月琴,周進欽,2019.0,B,Y,N,NaN,P
5502,3,A040,彩虹狗,高雄市林園區鳳林路一段169號,BC,NaN,特寵業字第Ｗ1070144號,2024/9/25 上午 12:00:00,吳慧珠,龔高益,2019.0,C,Y,N,NaN,P


In [11]:
# 執行ETL
# 去除不必要的欄位
df_filtered = df[["legalname", "legaladdress", "busitem", "animaltype", "state_flag"]]
df_filtered

,legalname,legaladdress,busitem,animaltype,state_flag
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,N
1,啟豐商行,桃園市桃園區文中一路25巷39號,AB,貓,N
2,偶然貓舍,臺南市永康區二王里41鄰南工街212巷3弄13號,ABC,貓,N
3,沐光貓舍,臺中市南區工學北路357號十樓之3,ABC,貓,N
4,森光貓境貓舍,新北市板橋區雙十路2段46巷22弄5號1樓,ABC,貓,N
...,...,...,...,...,...
5499,澎澎犬舍,高雄市旗山區大福巷5號,AB,狗,B
5500,東威犬舍,高雄市楠梓區藍昌路51號,ABC,狗,P
5501,台灣犬研育場,高雄市鳥松區神農路335-1號,ABC,狗,P
5502,彩虹狗,高雄市林園區鳳林路一段169號,BC,NaN,P


In [12]:
# busitem = C表示寄養，state_flag = N表示營業中
df_filtered = df_filtered[
    (df_filtered["busitem"].str.contains("C")) & (df_filtered["state_flag"] == "N")
]
df_filtered

,legalname,legaladdress,busitem,animaltype,state_flag
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,N
2,偶然貓舍,臺南市永康區二王里41鄰南工街212巷3弄13號,ABC,貓,N
3,沐光貓舍,臺中市南區工學北路357號十樓之3,ABC,貓,N
4,森光貓境貓舍,新北市板橋區雙十路2段46巷22弄5號1樓,ABC,貓,N
5,東森寵物雲股份有限公司花蓮玉里分公司,花蓮縣玉里鎮和平路96號,C,狗、貓,N
...,...,...,...,...,...
5368,假日貓坊,桃園市龍潭區百年里百年路33號,ABC,貓,N
5420,貓屋住宿用品店,雲林縣斗六市永安路77號,C,貓,N
5424,威地犬舍,雲林縣虎尾鎮安溪路121-6號,ABC,狗,N
5452,東森寵物雲股份有限公司南屯文心分公司,臺中市南屯區文心路一段459號,BC,狗、貓,N


In [13]:
# 找出屬於六都的機構
df_filtered["city"], df_filtered["district"] = zip(
    *df_filtered["legaladdress"].apply(ed.extract_city_district)
)
df_filtered = df_filtered[df_filtered["city"].notna()].reset_index(drop=True)
df_filtered

C:\Users\dinni\AppData\Local\Temp\ipykernel_408\4290206223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["city"], df_filtered["district"] = zip(
C:\Users\dinni\AppData\Local\Temp\ipykernel_408\4290206223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["city"], df_filtered["district"] = zip(


,legalname,legaladdress,busitem,animaltype,state_flag,city,district
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,N,臺北市,大安區
1,偶然貓舍,臺南市永康區二王里41鄰南工街212巷3弄13號,ABC,貓,N,臺南市,永康區
2,沐光貓舍,臺中市南區工學北路357號十樓之3,ABC,貓,N,臺中市,南區
3,森光貓境貓舍,新北市板橋區雙十路2段46巷22弄5號1樓,ABC,貓,N,新北市,板橋區
4,牛寶法鬥犬舍,高雄市林園區龍潭路225巷18號,ABC,狗,N,高雄市,林園區
...,...,...,...,...,...,...,...
2090,貓的旅坊,臺南市東區裕農路691號1-2樓,C,貓,N,臺南市,東區
2091,美樂犬莊,臺南市東山區東中里16鄰新東路二段50號,ABC,狗,N,臺南市,東山區
2092,假日貓坊,桃園市龍潭區百年里百年路33號,ABC,貓,N,桃園市,龍潭區
2093,東森寵物雲股份有限公司南屯文心分公司,臺中市南屯區文心路一段459號,BC,狗、貓,N,臺中市,南屯區


In [14]:
# 搜尋名字有旅、住、宿的資料
df_filtered = df_filtered[df_filtered["legalname"].str.contains("旅|住|宿", regex=True)]
df_filtered

,legalname,legaladdress,busitem,animaltype,state_flag,city,district
47,緹米貓旅館,新北市永和區福和路167號2樓,C,貓,N,新北市,永和區
53,瑞發財貓旅館,臺南市南區金華路一段468號1-2樓,BC,貓,N,臺南市,南區
58,吾悠喵旅生活館,新北市中和區中正路210號1樓,C,貓,N,新北市,中和區
66,樂牧寵物旅館,新北市新莊區建興街20號1樓,BC,狗、貓,N,新北市,新莊區
68,糖果屋貓咪旅館,新北市板橋區大觀路2段7巷24號,C,貓,N,新北市,板橋區
...,...,...,...,...,...,...,...
1997,狗狗打呼寵物旅館,臺中市西屯區臺中市西屯區青海路一段194號2樓,C,狗,N,臺中市,西屯區
2011,阿尼牧絲寵物旅館,臺中市烏日區臺中市烏日區高鐵路一段12號1樓、2樓、3樓,C,狗、貓,N,臺中市,烏日區
2048,貝果小貓旅店,臺中市烏日區環河北路258號,C,貓,N,臺中市,烏日區
2053,柴町寵物旅舍,桃園市桃園區中正里22鄰文中路10號,C,狗,N,桃園市,桃園區


In [15]:
# 只留下名稱、地址、市和區
df_filtered = df_filtered[["legalname", "legaladdress", "city", "district"]]
df_filtered

,legalname,legaladdress,city,district
47,緹米貓旅館,新北市永和區福和路167號2樓,新北市,永和區
53,瑞發財貓旅館,臺南市南區金華路一段468號1-2樓,臺南市,南區
58,吾悠喵旅生活館,新北市中和區中正路210號1樓,新北市,中和區
66,樂牧寵物旅館,新北市新莊區建興街20號1樓,新北市,新莊區
68,糖果屋貓咪旅館,新北市板橋區大觀路2段7巷24號,新北市,板橋區
...,...,...,...,...
1997,狗狗打呼寵物旅館,臺中市西屯區臺中市西屯區青海路一段194號2樓,臺中市,西屯區
2011,阿尼牧絲寵物旅館,臺中市烏日區臺中市烏日區高鐵路一段12號1樓、2樓、3樓,臺中市,烏日區
2048,貝果小貓旅店,臺中市烏日區環河北路258號,臺中市,烏日區
2053,柴町寵物旅舍,桃園市桃園區中正里22鄰文中路10號,桃園市,桃園區


In [16]:
# 修改欄位名稱
df_filtered = df_filtered.rename(
    columns={
        "legalname": "name",
        "legaladdress": "address",
    }
)
df_filtered

,name,address,city,district
47,緹米貓旅館,新北市永和區福和路167號2樓,新北市,永和區
53,瑞發財貓旅館,臺南市南區金華路一段468號1-2樓,臺南市,南區
58,吾悠喵旅生活館,新北市中和區中正路210號1樓,新北市,中和區
66,樂牧寵物旅館,新北市新莊區建興街20號1樓,新北市,新莊區
68,糖果屋貓咪旅館,新北市板橋區大觀路2段7巷24號,新北市,板橋區
...,...,...,...,...
1997,狗狗打呼寵物旅館,臺中市西屯區臺中市西屯區青海路一段194號2樓,臺中市,西屯區
2011,阿尼牧絲寵物旅館,臺中市烏日區臺中市烏日區高鐵路一段12號1樓、2樓、3樓,臺中市,烏日區
2048,貝果小貓旅店,臺中市烏日區環河北路258號,臺中市,烏日區
2053,柴町寵物旅舍,桃園市桃園區中正里22鄰文中路10號,桃園市,桃園區


In [ ]:
# 透過google api並傳送名稱與地址取得place_id
API_KEY = os.getenv("GOOGLE_MAP_KEY_CHGWYELLOW")
result = []

# enumerate會自動將被iterate的物件附上index
for i, (idx, row) in enumerate(df_filtered.iterrows()):
    query = f"{row['name']} {row['address']}"

    result.append(gm.get_place_id(API_KEY, query))
df_filtered["place_id"] = np.nan
df_filtered.loc[:, "place_id"] = result
df_filtered

C:\Users\dinni\AppData\Local\Temp\ipykernel_408\1597705559.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['ChIJcXJcgHSpQjQRNDlpI-2FZdI', 'ChIJY2gsl0J1bjQRImL6pDP8rY0', 'ChIJpfglnWcDaDQROqnauibQ4pE', 'ChIJWde5Zsvni2MRU7nv4bWTcgM', 'ChIJHcIvT6ypQjQRc9gESCt3ZSY', 'ChIJ6bxHpL6pQjQRxT3sIsWGHHQ', 'ChIJlZ49HAd3bjQRVI-hqM6Csec', 'ChIJp6ulzdofaDQRabB4RPtieSg', 'ChIJgXY55DQ9aTQRcHNGqavnT00', 'ChIJr8ISuASqQjQROYYmrkk_kdo', 'ChIJLVJZNjEdaDQRFv3zliGQvys', 'ChIJe1bS3SKoQjQRWbq1T-LgvD4', 'ChIJT6_obDYPbjQRD302Ipqcqyg', 'ChIJe4YFaACpQjQR2HKiVrJ87sE', 'ChIJhZk351Q8aTQR1K8s_YrCVMM', 'ChIJvapPIRYFbjQRYGFMsxcehgM', 'ChIJP5JrgUoiaDQRrpVQ4PapsD8', 'ChIJ4RQ6mX8FbjQRUsnNU65hy-Y', 'ChIJ561iVFQ8aTQRw8diMoHwU5w', 'ChIJvwYbBXQRaTQRY0y7pZgr-QY', 'ChIJqazxFIMfaDQRBrkmvDtYf7M', 'ChIJQ90SF30baDQRDhhCkm5QzU0', 'ChIJR4JtH42pQjQR0IsAAXV-kas', 'ChIJI4ikpxcXaTQRoPt_oh_Pgck', 'ChIJqVkEw-p7bjQRsoxoM4Sz28c', 'ChIJcwLVUV2rQjQRe_yzF_mMEdA', 'ChIJuYsH

In [ ]:
# 檢查是否有空值
df_filtered.isna().any()

name        False
address     False
city        False
district    False
place_id    False
dtype: bool

In [22]:
# 儲存ETL後CSV檔
path = "../../../data/processed/pet_hotel_ETL.csv"
sd.store_to_csv_no_index(df_filtered, path)

[✓] 不含index的CSV檔已存檔完畢
